In [ ]:
'''
import pandas as pd
import numpy as np
import glob

path = r'D://Research/DIT4BEARs/Road Measurements/Internship data/All' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

all_files
'''

In [ ]:
'''
frame.head()
'''

In [ ]:
'''
frame.shape
'''

In [ ]:
'''
frame.to_csv('D://Research/DIT4BEARs/Road Measurements/Internship data/UiTdata.csv')
'''

In [ ]:
#Import Important libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import plotly.express as px
from fbprophet.plot import plot_plotly, plot_components_plotly
from fbprophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose as sd
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import missingno as msno
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

In [ ]:
f = pd.read_csv('../input/road-dataset/UiTdata.csv')
f.head()

Data fields

Date - The date for recording (YYYY-MM-DD)\
Time - The time when recording at a given date (HH:MM:SS.6f)\
S1 - Measurement signal 1 (not to be used)\
S2 - Measurement signal 2 (not to be used)\
S3 - Measurement signal 3 (not to be used)\
Friction - Measure friction value between 0.1-0.81\
State - Values of 1-6 for: dry, moist, wet, icy, snowy, slushy\
Ta - Temperature in the air\
S7 - N/A\
Tsurf - Temperature on the measured surface\
S9 - N/A\
S10 - N/A\
S11 -N/A\
Water - Water layer thicknes in 0-3 mm\
Height - Above sealevel in m\
Tdew - The dewpoint temperature (not used in every aspect)\
Friction2 - Another calculation of friction (not used)\
Distance - Driven distance in m since startup of a session\
Serial(sensor RCM411) - Serial number of the current sensor 

In [ ]:
f.shape

In [ ]:
f.drop(columns=['Unnamed: 0','S1','S2','S3','S7','S9','S10','S11','Tdew','Friction2','Serial(RCM411 V 2.59 2016-09-29 - RCM Mobile v1.5.1)','Latitude','Longitude'], inplace=True, axis=1)
df = f
df.head(25)

In [ ]:
df2 = df['Date']
blankIndex=[''] * len(df2)
df2.index=blankIndex
data = str(df2).replace(".","-")
pd.DataFrame([x.split(';') for x in data.split('\n')])

In [ ]:
df2 = df['Date']
data = str(df2).replace(".","-")
pd.DataFrame([x for x in data.split('\n')]).head(15)

In [ ]:
df['Date']=pd.DataFrame([x.split(';') for x in data.split('\n')])
df.head(25)

In [ ]:
df.info()

In [ ]:
df.describe().transpose()

In [ ]:
df.isnull().sum()

In [ ]:
# Visualize our missing data

msno.bar(df, color="dodgerblue")
plt.show()

In [ ]:
df = df.dropna()
msno.bar(df, color="dodgerblue")
plt.show()

In [ ]:
df.drop_duplicates(subset=['Friction','Speed','Direction','Height'],keep='first',inplace=False,ignore_index=False)

In [ ]:
df

In [ ]:
df.iloc[:,2:18]

In [ ]:
Accident_rates1 = [0.8, 0.55, 0.25, 0.2]
Accident_rates = []
for x in df['Friction']:
    if x < 0.15:
        Accident_rates.append(0.8)
    elif (x>=0.15 and x<=0.24):
        Accident_rates.append(0.55)
    elif(x>=0.25 and x<=0.34):
        Accident_rates.append(0.25)
    elif(x>=0.35 and x<=0.45):
        Accident_rates.append(0.2)
    else:
        Accident_rates.append(0.05) #for preventive meaasures

In [ ]:
df['Accident_rates_friction'] = Accident_rates
df.tail()

In [ ]:
Accident_rates_st = []
for x in df['State']:
    if x == 1:
        Accident_rates_st.append(0.12)#Dry
    elif x == 2:
        Accident_rates_st.append(0.16)#Moist
    elif x == 3:
        Accident_rates_st.append(0.16)#Wet
    elif x == 4:
        Accident_rates_st.append(0.53)#Icy
    elif x == 5:
        Accident_rates_st.append(0.3) #Snowy
    elif x == 6:
        Accident_rates_st.append(0.18)#Slushy

In [ ]:
df['Accident_rates_state'] = Accident_rates_st

In [ ]:
month=[]
for x in df.iloc[:,0]:
    month.append(x.split('.')[1])
    #year,month,day=x.split('.')

In [ ]:
df['Month']=month

In [ ]:
df['Safety'] = df['Accident_rates_friction'] * df['Accident_rates_state']
df.head()

In [ ]:
np.random.seed(0)
import seaborn as sns
sns.set_theme()
plt.figure(figsize= (15,10))
_ = sns.heatmap(df.iloc[:,2:].corr(), annot= True, cmap= 'coolwarm')

In [ ]:
'''
df['Time(+01:00)'] = pd.to_datetime(df['Time(+01:00)'], format='%H:%M:%S').dt.hour
df.groupby(['Date','Time(+01:00)']).mean()
df = df.drop_duplicates()
df.head()
'''

In [ ]:
# DISTRIBUTION OF THE DEPENDENT VARIABLE

plt.figure(figsize=(20,5))
sns.distplot(df['State'], bins=40, kde=True, color='red')
plt.title('State Distribution')
plt.show()

In [ ]:
# States by different variables

fig, ax = plt.subplots(2, 2, figsize= (10,10))
ax[0,0].scatter(df['Friction'], df['State'])
ax[0,0].set_title('State by friction')
ax[0,1].scatter(df['Ta'], df['State'])
ax[0,1].set_title('State by Ta')
ax[1,0].scatter(df['Height'], df['State'])
ax[1,0].set_title('State by Height')
ax[1,1].scatter(df['Speed'], df['State'])
ax[1,1].set_title('State by Speed')
plt.show()

## Decomposing Time-Series data into Trend and Seasonality

In [ ]:
import math
ts= df.groupby("Date")["State"].mean().round(decimals=0)

In [ ]:
len(df.groupby("Date")["State"].mean())

In [ ]:
# Visualize Residuals, Seasonal, Trend, and level

res = sm.tsa.seasonal_decompose(ts.values,period=7,model="multiplicative")
res.plot()
plt.show()

In [ ]:
pd.date_range("2020-11-30","2021-03-03", freq="AS").tolist()
import datetime
datetime.ts[].isoformat()

In [ ]:
for i in range(64):
    ts_year = ts[i:i+1]
   # y=ts_year.values
    #x=ts_year.index.day
    print(ts_year.split())

In [ ]:
# Visualize the weekly sales by year
fig = go.Figure()
years = pd.date_range("2020-11-30","2021-03-03", freq="H").tolist() # range dates by year
for i in range(len(years)-1):
    ts_year = ts[years[i]:years[i+1]]
    fig.add_trace(
        go.Scatter(
            y=ts_year.values,
            x=ts_year.index.week,
            name=years[i].year,
        ))

fig.update_layout(
    title="weekly sales by year",
    xaxis_title="weeks",
    yaxis_title="sales",
    legend_title="year",
    yaxis_tickprefix = '$',
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ))
fig.show()

### Target Variable: State 
### Features: Ta, Water, Tsurf, Speed, Direction

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

X = df.loc[:,['Ta','Water','Tsurf','Speed','Direction']]
y = df['State']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
knn = KNeighborsClassifier(n_neighbors = 6)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

logreg = LogisticRegression()

c_space = np.logspace(-1,1,2)
param_grid = {'C': c_space}
logreg_cv = GridSearchCV(logreg, param_grid, cv=3)

logreg_cv.fit(X_train, y_train)

y_pred = logreg_cv.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

In [ ]:
from sklearn.preprocessing import scale
X_scaled = scale(X)
print("Mean of Scaled features {}".format(np.mean(X_scaled)))
print("Standard deviation of Scaled features {}".format(np.std(X_scaled)))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

steps = [('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier())]

pipeline = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline to the training set: knn_scaled
knn_scaled = pipeline.fit(X_train, y_train)

# Instantiate and fit a k-NN classifier to the unscaled data
knn_unscaled = KNeighborsClassifier().fit(X_train, y_train)

# Compute and print metrics
print('Accuracy with Scaling: {}'.format(knn_scaled.score(X_test, y_test)))
print('Accuracy without Scaling: {}'.format(knn_unscaled.score(X_test, y_test)))

In [ ]:
from sklearn.svm import SVC
steps = [('scaler', StandardScaler()),
         ('SVM', SVC())]

pipeline = Pipeline(steps)

# Specify the hyperparameter space
parameters = {'SVM__C':[1, 10, 100],
              'SVM__gamma':[0.1, 0.01]}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=21)

# Instantiate the GridSearchCV object: cv
cv = GridSearchCV(pipeline, param_grid=parameters)

# Fit to the training set
cv.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

# Compute and print metrics
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

### Target Variable: State 
### Features: Ta, Water, Tsurf, Speed, Direction, Height, Friction

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

X = df.loc[:,['Ta','Water','Tsurf','Speed','Direction','Height','Friction']]
y = df['State']

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

steps = [('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier())]

pipeline = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline to the training set: knn_scaled
knn_scaled = pipeline.fit(X_train, y_train)

# Instantiate and fit a k-NN classifier to the unscaled data
knn_unscaled = KNeighborsClassifier().fit(X_train, y_train)

# Compute and print metrics
print('Accuracy with Scaling: {}'.format(knn_scaled.score(X_test, y_test)))
print('Accuracy without Scaling: {}'.format(knn_unscaled.score(X_test, y_test)))

In [ ]:
from sklearn.svm import SVC
steps = [('scaler', StandardScaler()),
         ('SVM', SVC())]

pipeline = Pipeline(steps)

# Specify the hyperparameter space
parameters = {'SVM__C':[100],
              'SVM__gamma':[0.1]}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=21)

# Instantiate the GridSearchCV object: cv
cv = GridSearchCV(pipeline, param_grid=parameters)

# Fit to the training set
cv.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

# Compute and print metrics
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

In [ ]:
### Target Variable: State 
### Features: Ta, Water, Tsurf, Speed, Direction, Height, Friction
Accuracy with Scaling: 0.93762372198151
Accuracy without Scaling: 0.6891010419981559

### Target Variable: Safety = Accident rate in case of state * Accident rate in case of friction coefficient

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

X = df.loc[:,['Ta','Water','Tsurf','Speed','Direction','Height']]
y = df['Safety']

In [ ]:
from sklearn.linear_model import ElasticNet
steps = [('scaler', StandardScaler()),
         ('elasticnet', ElasticNet())]

# Create the pipeline: pipeline 
pipeline = Pipeline(steps)

# Specify the hyperparameter space
parameters = {'elasticnet__l1_ratio':np.linspace(0,1,30)}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create the GridSearchCV object: gm_cv
gm_cv = GridSearchCV(pipeline, param_grid=parameters)

# Fit to the training set
gm_cv.fit(X_train, y_train)

# Compute and print the metrics
r2 = gm_cv.score(X_test, y_test)
print("Tuned ElasticNet Alpha: {}".format(gm_cv.best_params_))
print("Tuned ElasticNet R squared: {}".format(r2))